In [1]:
%load_ext autoreload
%matplotlib inline
%autoreload 2

In [2]:
# import the needed modules

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import datetime

from finrl.apps import config
from data_loader import DataLoader
from trainer import Trainer

matplotlib.use('Agg')

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


In [3]:
def create_folders():
    if not os.path.exists("./" + config.DATA_SAVE_DIR):
        os.makedirs("./" + config.DATA_SAVE_DIR)
    if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
        os.makedirs("./" + config.TRAINED_MODEL_DIR)
    if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
        os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
    if not os.path.exists("./" + config.RESULTS_DIR):
        os.makedirs("./" + config.RESULTS_DIR)

In [4]:
"""
    The main script used for training the DRL model

    Can adjust the train and trade (used for backtesting) dates.

    Also it is possible to specify multiple stocks the DRL has to trade. 

"""

# First created the folders
create_folders()

# Adjust these dates accordingly
train_dates = ('2009-01-01','2019-01-01')
trade_dates = ('2019-01-01','2021-11-24')

# Can extend the ticker list with stocks listed in config.DOW_30_TICKER
ticker_list = ['AAPL']

# Initialize DataLoader object and preprocess the historical stock data, then split the data
dl = DataLoader(ticker_list=ticker_list, start_date=train_dates[0], end_date=trade_dates[1])
dl.preprocess()

df = dl.get_data()

dl.split_dataset(train_dates, trade_dates)

train_data = dl.get_train_data()
trade_data = dl.get_trade_data()


# Create a Trainer object which creates the Agent and train&trade environments
model = Trainer(train_data, trade_data)

model.set_environment('train')
model.set_environment('trade')
model.set_agent()

# Train the model
model.train()

# Save the model
model.save_model('agent.model')

print("Done")


Fetching data....
2009-01-01 2021-11-24 ['AAPL']
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (3248, 8)
Starting preprocessing...
Successfully added technical indicators
Splitting dataset into train and trade
Creating training environment
caching data
data cached!
Creating trading environment
caching data
data cached!
{'n_steps': 256, 'ent_coef': 0.0, 'learning_rate': 5e-06, 'batch_size': 1024, 'gamma': 0.99}


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/stable_baselines3/ppo/ppo.py:137: UserWarning: You have specified a mini-batch size of 1024, but because the `RolloutBuffer` is of size `n_steps * n_envs = 256`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 256
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=256 and n_envs=1)
  warnings.warn(


INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
EPISODE|STEPS|TERMINAL_REASON|CASH           |TOT_ASSETS     |TERMINAL_REWARD_unsc|GAINLOSS_PCT|CASH_PROPORTION
   1| 499|update         |$980,515       |$996,933       |-0.00062% |-0.30666% |98.35%    
EPISODE|STEPS|TERMINAL_REASON|CASH           |TOT_ASSETS     |TERMINAL_REWARD_unsc|GAINLOSS_PCT|CASH_PROPORTION


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


   1| 225|Last Date      |$1,000,000     |$1,000,000     |0.00000%  |0.00000%  |100.00%   
Eval num_timesteps=500, episode_reward=0.00 +/- 0.00
Episode length: 226.00 +/- 0.00
New best mean reward!
   1| 999|update         |$1,006,100     |$1,006,100     |0.00061%  |0.61001%  |100.00%   
   3| 225|Last Date      |$1,000,000     |$1,000,000     |0.00000%  |0.00000%  |100.00%   
Eval num_timesteps=1000, episode_reward=0.00 +/- 0.00
Episode length: 226.00 +/- 0.00
   1|1499|update         |$1,003,663     |$1,003,663     |0.00024%  |0.36630%  |100.00%   
   5| 225|Last Date      |$1,000,000     |$1,000,000     |0.00000%  |0.00000%  |100.00%   
Eval num_timesteps=1500, episode_reward=0.00 +/- 0.00
Episode length: 226.00 +/- 0.00
   1|1574|Last Date      |$999,271       |$1,003,038     |0.00019%  |0.30383%  |99.62%    
   7| 225|Last Date      |$1,000,000     |$1,000,000     |0.00000%  |0.00000%  |100.00%   
Eval num_timesteps=2000, episode_reward=0.00 +/- 0.00
Episode length: 226.00 +/- 0.0

In [6]:
"""
    Backtest the performance of the trained model
"""

plot = model.backtest(trade_dates)

Starting backtesting
EPISODE|STEPS|TERMINAL_REASON|CASH           |TOT_ASSETS     |TERMINAL_REWARD_unsc|GAINLOSS_PCT|CASH_PROPORTION
   1| 225|Last Date      |$999,748       |$1,004,763     |0.00212%  |0.47625%  |99.50%    
hit end!
Annual return          0.005336
Cumulative returns     0.004763
Annual volatility      0.006040
Sharpe ratio           0.887974
Calmar ratio           1.402499
Stability              0.688456
Max drawdown          -0.003804
Omega ratio            1.186973
Sortino ratio          1.378179
Skew                        NaN
Kurtosis                    NaN
Tail ratio             1.131028
Daily value at risk   -0.000740
dtype: float64
[*********************100%***********************]  1 of 1 completed
Shape of DataFrame:  (227, 8)


Start date,2021-01-05
End date,2021-11-23
Total months,10
,Backtest
Annual return,0.534%
Cumulative returns,0.476%
Annual volatility,0.604%
Sharpe ratio,0.89
Calmar ratio,1.4
Stability,0.69
Max drawdown,-0.38%


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,0.38,2021-02-05,2021-03-09,2021-06-18,96
1,0.23,2021-09-08,2021-09-21,NaT,NaN
2,0.18,2021-07-15,2021-07-20,2021-07-27,9
3,0.14,2021-07-27,2021-07-29,2021-08-16,15
4,0.13,2021-08-17,2021-08-19,2021-08-31,11


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/plotting/_matplotlib/tools.py:390: MatplotlibDeprecationWarning: 
The is_last_row function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use ax.get_subplotspec().is_last_row() instead.
  if ax.is_last_row():
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/plotting/_matplotlib/tools.py:400: MatplotlibDeprecationWarning: 
The is_first_col function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use ax.get_subplotspec().is_first_col() instead.
  if ax.is_first_col():
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/plotting/_matplotlib/tools.py:390: MatplotlibDeprecationWarning: 
The is_last_row function was deprecated in Matplotlib 3.4 and will be removed two minor releases later. Use ax.get_subplotspec().is_last_row() instead.
  if ax.is_last_row():
/Library/Frameworks

Stress Events,mean,min,max
New Normal,0.00%,-0.12%,0.16%


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexes/base.py:5277: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  start_slice, end_slice = self.slice_locs(start, end, step=step, kind=kind)
